# ทำนายโอกาสการผิดนัดชำระหนี้ของลูกค้าบัตรเครดิตในไต้หวันโดยโมเดล Random Forest

In [61]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix

In [63]:
from sklearn.preprocessing import StandardScaler

In [65]:
from sklearn.metrics import recall_score

In [67]:
from sklearn.datasets import make_classification

In [69]:
from sklearn.model_selection import cross_val_score

In [71]:
import optuna

In [73]:
import matplotlib.pyplot as plt

In [75]:
from sklearn.metrics import accuracy_score

In [95]:
from imblearn.combine import SMOTEENN 

In [99]:
from collections import Counter

# 1. จัดเตรียมข้อมูลเพื่อใช้ในการวิเคราะห์

In [77]:
df = pd.read_csv('/Users/1hundredeyes/default_of_credit_card_clients(1).csv')

ID: รหัสของลูกค้าแต่ละราย

LIMIT_BAL: วงเงินบัตรเครดิต(ดอลลาร์ไต้หวันใหม่); (รวมถึงสินเชื่อส่วนบุคคลและครอบครัว/เครดิตเสริม)

SEX: เพศ (1 = ชาย, 2 = หญิง)

EDUCATION: (1 = บัณฑิตวิทยาลัย, 2 = มหาวิทยาลัย, 3 = มัธยมปลาย, 4 = อื่นๆ, 0 = ไม่ทราบค่า, 5 = ไม่ทราบค่า, 6 = ไม่ทราบค่า)

MARRIAGE: Marital status (1 = แต่งงานแล้ว, 2 = โสด, 3 = อื่นๆ, 0 = ไม่ทราบค่า)

AGE: อายุ(ปี)

PAY_0: สถานะการชำระคืนในเดือนกันยายน พ.ศ. 2548 (-2 = ไม่มีการเคลื่องไหว, -1 = ชำระตามกำหนด, 0 = การใช้เครดิตหมุนเวียน(ชำระขั้นต่ำเท่านั้น), 1 = การชำระเงินล่าช้าเป็นเวลาหนึ่งเดือน, 2 = การชำระเงินล่าช้าเป็นเวลาสองเดือน, ... 8 = การชำระเงินล่าช้าเป็นเวลาแปดเดือน, 9 = การชำระเงินล่าช้าเป็นเวลาเก้าเดือน)

PAY_2: สถานะการชำระคืนในเดือนสิงหาคม พ.ศ. 2548 (ขนาดเช่นเดียวกับข้างต้น)

PAY_3: สถานะการชำระคืนในเดือนกรกฎาคม พ.ศ. 2548 (ขนาดเช่นเดียวกับข้างต้น)

PAY_4: สถานะการชำระคืนในเดือนมิถุนายน พ.ศ. 2548 (ขนาดเช่นเดียวกับข้างต้น)

PAY_5: สถานะการชำระคืนในเดือนพฤษภาคม พ.ศ. 2548 (ขนาดเช่นเดียวกับข้างต้น)

PAY_6: สถานะการชำระคืนในเดือนเมษายน พ.ศ. 2548 (ขนาดเช่นเดียวกับข้างต้น)

BILL_AMT1: จำนวนเงินในใบแจ้งยอดในเดือนกันยายน พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

BILL_AMT2: จำนวนเงินในใบแจ้งยอดในเดือนสิงหาคม พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

BILL_AMT3: จำนวนเงินในใบแจ้งยอดในเดือนกรกฎาคม พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

BILL_AMT4: จำนวนเงินในใบแจ้งยอดในเดือนมิถุนายน พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

BILL_AMT5: จำนวนเงินในใบแจ้งยอดในเดือนพฤษภาคม พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

BILL_AMT6: สถานะการชำระคืนในเดือนเมษายน พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

PAY_AMT1: จำนวนเงินที่ชำระครั้งก่อนในเดือนกันยายน พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

PAY_AMT2: จำนวนเงินที่ชำระครั้งก่อนในเดือนสิงหาคม พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

PAY_AMT3: จำนวนเงินที่ชำระครั้งก่อนในเดือนกรกฎาคม พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

PAY_AMT4: จำนวนเงินที่ชำระครั้งก่อนในเดือนมิถุนายน พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

PAY_AMT5: จำนวนเงินที่ชำระครั้งก่อนในเดือนพฤษภาคม พ.ศ. 2548 (ดอลลาร์ไต้หวันใหม่)

PAY_AMT6: จำนวนเงินที่ชำระครั้งก่อนในเดือนเมษายน พ.ศ. 25485 (ดอลลาร์ไต้หวันใหม่)

defaul payment next month: สถานะการชำระเงิน (1 = มีสถานะผิดนัดชำระหนี้, 0 = ไม่มีสถานะผิดนัดชำระหนี้)

* แก้ไข Index ให้เป็น ID เพื่อให้สะดวกต่อการวิเคราะห์

In [79]:
df.set_index("ID", inplace = True)

In [17]:
df.head(5)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


* ตรวจสอบค่า Null ในแต่ละฟีเจอร์

In [525]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 1 to 30000
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   LIMIT_BAL                   30000 non-null  int64
 1   SEX                         30000 non-null  int64
 2   EDUCATION                   30000 non-null  int64
 3   MARRIAGE                    30000 non-null  int64
 4   AGE                         30000 non-null  int64
 5   PAY_0                       30000 non-null  int64
 6   PAY_2                       30000 non-null  int64
 7   PAY_3                       30000 non-null  int64
 8   PAY_4                       30000 non-null  int64
 9   PAY_5                       30000 non-null  int64
 10  PAY_6                       30000 non-null  int64
 11  BILL_AMT1                   30000 non-null  int64
 12  BILL_AMT2                   30000 non-null  int64
 13  BILL_AMT3                   30000 non-null  int64
 14  BILL_AMT4  

จากการตรวจสอบข้างต้นไม่พบข้อมูล Null ในตัวแปรใดๆของข้อมูลการผิดนัดชำระของลูกค้าบัตรเครดิตในไต้หวัน

* ตรวจสอบข้อมูลในตัวแปร EDUCATION และ MARRIAGE

In [26]:
df['EDUCATION'].value_counts().sort_index()

EDUCATION
0       14
1    10585
2    14030
3     4917
4      123
5      280
6       51
Name: count, dtype: int64

In [28]:
df['MARRIAGE'].value_counts().sort_index()

MARRIAGE
0       54
1    13659
2    15964
3      323
Name: count, dtype: int64

จากตัวแปรอิระ EDUCATION พบค่าที่ไม่สามารถอธิบายความหมายของข้อมูล ได้แก่ 0, 5, 6 และตัวแปรอิระ MARRIAGE พบค่าที่ไม่สามารถอธิบายความหมายของข้อมูล ได้แก่ 0

* ซึ่งผู้วิจัยได้ทำการแก้ไขปัญหาดังกล่าว โดยแทนค่า 0, 5, 6 ที่มาจากตัวแปรอิระ EDUCATION เป็น 'others' และแทนค่า 0 ที่มาจากตัวแปรอิระ MARRIAGE เป็น 'others' เช่นเดียวกัน

In [81]:
df['EDUCATION'] = df['EDUCATION'].replace({0: 4, 5: 4, 6: 4})

In [83]:
df['MARRIAGE'] = df['MARRIAGE'].replace({0: 3})

* เปลี่ยนชื่อตัวแปรตาม default payment next month เป็น DEFAULT เพื่อให้สะดวกในการวิเคราะห์ข้อมูล

In [85]:
df = df.rename(columns = {'PAY_0': 'PAY_1'})

In [87]:
df = df.rename(columns = {'default payment next month': 'DEFAULT'})

* เนื่องจากข้อมูลมีจำนวน class 1 และ 0 ที่แตกต่างกันมาก จึงต้องทำการปรับสมดุลข้อมูลด้วยเทคนิคผสม (SMOTE + Undersampling)

In [89]:
x = df.drop('DEFAULT',axis=1)

In [91]:
y = df['DEFAULT']

In [101]:
smote_enn = SMOTEENN(random_state=101)
x_resampled, y_resampled = smote_enn.fit_resample(x, y)

print(Counter(y_resampled))

Counter({1: 17031, 0: 10394})


* แบ่งข้อมูลที่ใช้ในการวิเคราะห์ออกเป็น 2 ส่วน โดยในส่วนแรกเป็นข้อมูลที่ใช้ในการสร้างโมเดล(Traning Data) ซึ่งเป็นร้อยละ 70 ของข้อมูลทั้งหมด และในส่วนที่สองเป็นข้อมูลที่ใช้ในการทดสอบประสิทธิภาพในการทำนายค่าของโมเดล(Testing Data) ซึ่งเป็นร้อยละ 30 ของข้อมูลทั้งหมด

In [103]:
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.30, random_state=101, stratify = y_resampled)

* เนื่องจากข้อมูลในแต่ละฟีเจอร์มีขนาดที่แตกต่างกันมาก ผู้วิจัยจึงทำการปรับขนาดของฟีเจอร์โดยใช้วิธี StandardScaler

In [105]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [107]:
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)

# 2.) สร้างโมเดล Random Forest โดยใช้ Bayesian optimization ในการคัดเลือกไฮเปอร์พารามิเตอร์

* ความหมายของไฮเปอร์พารามิเตอร์แต่ละตัวในโมเดล Random Forest

1.) n_estimators คือ จำนวนต้นไม้ (trees) ที่จะสร้างใน Random Forest ยิ่งมีจำนวนต้นไม้มากขึ้น โมเดลมักจะมีความแม่นยำสูงขึ้น แต่ก็อาจทำให้ใช้เวลาฝึกฝนมากขึ้นและต้องการหน่วยความจำมากขึ้นด้วย

2.) max_depth คือ ความลึกสูงสุดของต้นไม้แต่ละต้น ถ้ากำหนดความลึกให้สูงเกินไป โมเดลอาจเกิดการ overfitting ซึ่งหมายถึงการเรียนรู้รายละเอียดมากเกินไปจากข้อมูลฝึกฝนจนไม่สามารถทั่วไปกับข้อมูลใหม่ได้ ในทางกลับกัน ถ้ากำหนดให้ต่ำเกินไป โมเดลอาจไม่สามารถจับความสัมพันธ์ที่ซับซ้อนในข้อมูลได้

3.) min_samples_split คือ จำนวนตัวอย่างขั้นต่ำที่ต้องมีในโหนดก่อนที่จะสามารถแบ่งเป็นโหนดใหม่ได้ ถ้าค่านี้สูงเกินไป โมเดลอาจไม่แบ่งโหนดซึ่งอาจทำให้การจำแนกประเภทไม่แม่นยำขึ้น ในขณะที่ถ้าต่ำเกินไป โมเดลอาจเรียนรู้รายละเอียดเล็กๆ ที่ไม่สำคัญจากข้อมูล

4.) min_samples_leaf คือ จำนวนตัวอย่างขั้นต่ำที่ต้องมีในโหนดใบ (leaf node) ถ้าค่านี้สูงเกินไป โมเดลจะทำให้โหนดใบมีจำนวนตัวอย่างมากขึ้น ซึ่งอาจช่วยลดการ overfitting ในทางกลับกัน ถ้าค่าต่ำเกินไป โหนดใบอาจมีเพียงตัวอย่างเดียว ซึ่งอาจทำให้เกิดการ overfitting ได้

5.) max_features คือ จำนวนฟีเจอร์สูงสุดที่จะพิจารณาสำหรับการแบ่งในแต่ละโหนด สามารถเลือกได้หลายวิธี เช่น การเลือกฟีเจอร์แบบสุ่ม (sqrt, log2) หรือการระบุสัดส่วนของฟีเจอร์ที่ใช้ (0.5, 0.75) การกำหนดค่าที่เหมาะสมจะช่วยให้โมเดลมีความหลากหลายและลดความเสี่ยงของการ overfitting


In [111]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 600)  
    max_depth = None  
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)  
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None, 0.5, 0.75])
    
    rf_model = RandomForestClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf,
        max_features = max_features,
        n_jobs = -1,
        random_state = None
    )

    score = cross_val_score(rf_model, x_train_scaled, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    
    return score.mean()  

In [113]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-02-27 20:41:17,107] A new study created in memory with name: no-name-9c668b13-bc90-4012-a8f0-0c489202fad0
[I 2025-02-27 20:41:25,669] Trial 0 finished with value: 0.866697870539203 and parameters: {'n_estimators': 140, 'min_samples_split': 11, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.866697870539203.
[I 2025-02-27 20:41:43,503] Trial 1 finished with value: 0.8646141437657378 and parameters: {'n_estimators': 170, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': 0.5}. Best is trial 0 with value: 0.866697870539203.
[I 2025-02-27 20:41:50,451] Trial 2 finished with value: 0.8550814556742207 and parameters: {'n_estimators': 154, 'min_samples_split': 17, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.866697870539203.
[I 2025-02-27 20:41:56,572] Trial 3 finished with value: 0.8712822213901189 and parameters: {'n_estimators': 135, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'log2'}. Bes

In [115]:
best_params = study.best_params
print("Best parameters:", best_params)

Best parameters: {'n_estimators': 309, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}


In [117]:
best_model = RandomForestClassifier(**best_params)
best_model.fit(x_train_scaled, y_train)

RandomForestClassifier(min_samples_split=3, n_estimators=309)

In [50]:
print(best_model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 10, 'min_samples_split': 6, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 334, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


จากการสร้างโมเดล Random Forest โดยใช้ Bayesian optimization ในการคัดเลือกไฮเปอร์พารามิเตอร์ที่เหมาะสมมากที่สุด พบว่ามีไฮเปอร์พารามิเตอร์ที่อยู่ในโมเดล Random Forest ได้แก่ n_estimators มีค่าเท่า 309, max_depth มีค่าเท่ากับ None, min_samples_split มีค่าเท่ากับ 6, min_samples_leaf มีค่าเท่ากับ 10, และ max_features มีค่าเท่ากับ log2

# 3.) ทดสอบประสิทธิภาพในการทำนายค่าของโมเดล Random Forest

In [121]:
predict = best_model.predict(x_test_scaled)

In [123]:
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86      3118
           1       0.91      0.93      0.92      5110

    accuracy                           0.90      8228
   macro avg       0.90      0.89      0.89      8228
weighted avg       0.90      0.90      0.90      8228



จากผลการทดสอบประสิทธิภาพในการทำนายโอกาสการผิดนัดชำระหนี้ของลูกค้าบัตรเครดิตในไต้หวันโดยโมเดล Random Forest พบว่ามีค่า Accuracy เท่ากับ 0.90 หมายความว่าโมเดลมีประสิทธิภาพในการทำนายที่แม่นยำ 90%

และถ้าหากวิเคราะห์จากค่า Precision และค่า Recall ของโอกาสที่ลูกค้าจะผิดนัดชำระหนี้บัตรเครดิต และโอกาสที่ลูกค้าจะไม่ผิดนัดชำระหนี้บัตรเครดิต พบว่าโมเดล Random Forest มีความสามารถในการทำนายค่าของกลุ่มลูกค้าที่ไม่ผิดนัดชำระหนี้บัตรเครดิตได้แม่นยำ 88% และมีความสามารถในการ Recall ของกลุ่มลูกค้าที่ไม่ผิดนัดชำระหนี้บัตรเครดิต 85% และมีความสามารถในการทำนายค่าของกลุ่มลูกค้าที่มีการผิดนัดชำระหนี้บัตรเครดิตได้แม่นยำ 91% และมีความสามารถในการ Recall ของกลุ่มลูกค้าที่มีการผิดนัดชำระหนี้บัตรเครดิต 93%